# Titanic: Machine Learning from Disaster
---


## Frame the problem
---
### Background
---
From the kaggle challenge page:
> The sinking of the Titanic is one of the most infamous shipwrecks in history.
>
> On April 15, 1912, during her maiden voyage, the widely considered “unsinkable” RMS Titanic sank after colliding with an iceberg. Unfortunately, there weren’t enough lifeboats for everyone onboard, resulting in the death of 1502 out of 2224 passengers and crew.
>
>While there was some element of luck involved in surviving, it seems some groups of people were more likely to survive than others.


>In this challenge, we ask you to build a predictive model that answers the question: “what sorts of people were more likely to survive?” using passenger data (ie name, age, gender, socio-economic class, etc).

### Dataset
---
Below is a table containing the contents of the dataset named `train.csv`:

| Variable  | Definition                                 	| Key                                            	|
|-----------|:----------------------------------------------|:--------------------------------------------------|
| Survived  | Survival                                   	| 0 = No, 1 = Yes                                	|
| Pclass   	| Ticket class                               	| 1 = 1st, 2 = 2nd, 3 = 3rd                      	|
| Sex      	| Sex                                        	|                                                	|
| Age      	| Age in years                               	|                                                	|
| SibSp    	| # of siblings / spouses aboard the Titanic 	|                                                	|
| Parch    	| # of parents / children aboard the Titanic 	|                                                	|
| Ticket   	| Ticket number                              	|                                                	|
| Fare     	| Passenger fare                             	|                                                	|
| Cabin    	| Cabin number                               	|                                                	|
| Embarked 	| Port of Embarkation                        	| C = Cherbourg, Q = Queenstown, S = Southampton 	|

## Set-up
---
Here, we set up all the paths and libraries used in the program, as well as important variables

In [57]:
import os,sys

globalT3Mbin = os.path.abspath(os.path.join(os.getcwd(),'..','..','bin'))
round1bin = os.path.abspath(os.path.join(os.getcwd(),'..','bin'))
if not (globalT3Mbin in sys.path):
    sys.path.append(globalT3Mbin)
if not (round1bin in sys.path):
    sys.path.append(round1bin)

In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC,SVC
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from utilities import splitter

ModuleNotFoundError: No module named 'pandas'

In [59]:
from pathlib import Path
data_folder_path = Path("../data")
training_set_path = data_folder_path / "train.csv"


In [60]:
seed = 1
plt.style.use('dark_background')
from sklearn import set_config
set_config(display='diagram')

NameError: name 'plt' is not defined

## Gather Data
---


In [61]:
data_set = pd.read_csv(training_set_path)
feature_names = data_set.columns.values
label_names = ['Survived']
print("The names of the features are - " +str(feature_names))



NameError: name 'pd' is not defined

## Nature of Data
---
Here we investigate certain properties of the data which will affect the algorithm. This includes missing data, range of values, specialy types of values, what features are categorical etc.


In [62]:
print(data_set.info())
print(data_set.head().to_string())

NameError: name 'data_set' is not defined

There are a hanful of **missing** data values for the categories **Age, Embarked and Cabin**. 


In [63]:
print("The number of records missing ages is {0} and the number with missing ages and cabins is {1}"
    .format(data_set.Age.isnull().sum(),(data_set.Age.isnull() & data_set.Cabin.isnull()).sum()))
#TODO interpret this 



NameError: name 'data_set' is not defined

## Data Insights
---
### Distribution of features
First, let us observe what the dsitribution of features is.

In [64]:
from pandas.plotting import scatter_matrix
data_set.hist(bins=20,figsize=(15,15))
plt.show()
scatter_matrix(data_set[["Age","Fare","Pclass","PassengerId"]],figsize=(15,15))
plt.show()

ModuleNotFoundError: No module named 'pandas'

#### Some interesting thoughts:

1. There is no apparent relation between the PassengerId and Fare/Class. This could be for several reasons. Firstly, PassengerId could be assigned by the collectors of the dataset to identitify all passengers. Secondly, it could be because there is no apparent link between when the tickets were purchased and who got them. For now, I would like to conclude that there is no relation between "Fare" and "Pclass".
2. Interestingly, the cost of a certain class ticket is not the same for everyone. This is reflected in the range of fairs associated with each class. After further investigation, it is found that the better the class, the higher the fair and thus the larger the variation in cost. Also, it is interesting to see the imbalance in concentrations of age in the class.
3. There does seem to be a connection between PClass and Age. The younger the person is, the less wealthy, and hence the lower the class. 

In [65]:
g = sns.FacetGrid(data_set,col="Pclass",size=6,xlim=(0,80),ylim=(0,600))
g = g.map(sns.kdeplot,"Age","Fare")
g.add_legend()
plt.show()
f,axes = plt.subplots(1,2,figsize=(15,5))
f = sns.boxplot(data=data_set,x='Age',y='Pclass',orient='h',ax=axes[0])
f = sns.boxplot(data=data_set,x='Fare',y='Pclass',orient='h',ax=axes[1])
plt.show()

NameError: name 'sns' is not defined

### Correlation
Let us check what the correlation between different features is:

In [66]:
correlation_matrix = data_set.corr()
plt.figure(figsize=(15,15))
sns.heatmap(correlation_matrix,annot=True,vmin=-1,vmax=1,cmap='YlGnBu')
plt.show()

NameError: name 'data_set' is not defined

Lets look at the features which are most correlated:

1. **Pclass and Fare** - Naturally, the higher the class (i.e. smaller the number), the more expensive it is.
2. **SibSp and Parch** - This makes sense. If you have a spouse on board, then your children might be one as well, and if you have siblings on board, then you are likely to be a child and have your parents on board

As PClass is weakly correlated with age, I decided to check survival rates by passenger class of different age groups.

In [67]:
data_set
men = data_set.loc[data_set.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

NameError: name 'data_set' is not defined

In [68]:
print(type(data_set))
print(sum(data_set['Survived']), ' survived / ', len(data_set), ' total passengers\n\n')
mclasscount, mclasssurv = np.zeros(3), np.zeros(3)
fclasscount, fclasssurv = np.zeros(3), np.zeros(3)
magecount, magesurv = np.zeros(4), np.zeros(4)
fagecount, fagesurv = np.zeros(4), np.zeros(4)
for i in range (len(data_set)):
    if data_set['Sex'][i] == 'male':
        temp_class = data_set['Pclass'][i] - 1
        mclasscount[temp_class] = mclasscount[temp_class] + 1
        mclasssurv[temp_class] = mclasssurv[temp_class] + data_set['Survived'][i]

        if data_set['Age'][i] <= 16:
            magecount[0] = magecount[0] + 1
            magesurv[0] = magesurv[0] + data_set['Survived'][i]
        elif data_set['Age'][i] <= 25:
            magecount[1] = magecount[1] + 1
            magesurv[1] = magesurv[1] + data_set['Survived'][i]
        elif data_set['Age'][i] <= 40:
            magecount[2] = magecount[2] + 1
            magesurv[2] = magesurv[2] + data_set['Survived'][i]
        else:
            magecount[3] = magecount[3] + 1
            magesurv[3] = magesurv[3] + data_set['Survived'][i]

    elif data_set['Sex'][i] == 'female':
        temp_class = data_set['Pclass'][i] - 1
        fclasscount[temp_class] = fclasscount[temp_class] + 1
        fclasssurv[temp_class] = fclasssurv[temp_class] + data_set['Survived'][i]

        if data_set['Age'][i] <= 16:
            fagecount[0] = fagecount[0] + 1
            fagesurv[0] = fagesurv[0] + data_set['Survived'][i]
        elif data_set['Age'][i] <= 25:
            fagecount[1] = fagecount[1] + 1
            fagesurv[1] = fagesurv[1] + data_set['Survived'][i]
        elif data_set['Age'][i] <= 40:
            fagecount[2] = fagecount[2] + 1
            fagesurv[2] = fagesurv[2] + data_set['Survived'][i]
        else:
            fagecount[3] = fagecount[3] + 1
            fagesurv[3] = fagesurv[3] + data_set['Survived'][i]

#print('\nMale survival rate by class', mclasssurv / mclasscount)
#print('\nFemale survival rate by class', fclasssurv / fclasscount)

pclass_sex_array = pd.DataFrame([mclasssurv/mclasscount, fclasssurv/fclasscount], columns = ['1','2','3'], index = ['M', 'F']);
print('Survival rate class/sex\n', pclass_sex_array, '\n\n')

age_sex_array = pd.DataFrame([magesurv/magecount, fagesurv/fagecount], columns = ['0-16','17-25','26-40','41+'], index=['M','F']);
print('Survival rate age/sex\n', age_sex_array)

NameError: name 'data_set' is not defined

In [69]:
plt.hist([pclass_sex_array.loc['M'],pclass_sex_array.loc['F']],color=['b','r'], alpha=0.5,density=True,label=['M','F']) #Need better may of visuallizing
plt.legend()
plt.title("Survival Rate base on class and sex")
plt.show()

NameError: name 'plt' is not defined

### Visualizing the results:
---

In [70]:
def plot_features(Xfeature,YFeature,X,Y):
    plt.scatter(X[Xfeature],X[YFeature],c=Y.values)
    plt.show()
plot_features("Age","Pclass",data_set_X,data_set_Y)

NameError: name 'data_set_X' is not defined

## Preparing data for algorithm
These are some initial steps taken to pre-process data.

---
### Remove unwanted features
I believe that **PassangerID** and **Name** will not be useful at the moment. Likewise, the **ticket** will not be used as lots of processing is required on it, and neither will **cabin** as only 1/4 of records have this information.

In [71]:
from transformers.FeatureSelector import FeatureSelector
cat_selector = FeatureSelector(['Sex','Embarked', 'Pclass'])
num_selector = FeatureSelector(['Age','SibSp','Parch','Fare'])# should pclass be numerical or categorical? - I think it should be categorical (NN)
cat_selector.fit_transform(data_set)

ModuleNotFoundError: No module named 'sklearn'

### Fix missing values

In [72]:
from sklearn.impute import SimpleImputer
cat_imputer = SimpleImputer(strategy='most_frequent')
num_imputer = SimpleImputer(strategy='mean')

ModuleNotFoundError: No module named 'sklearn'

### Encoding categorical data

In [73]:
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()

ModuleNotFoundError: No module named 'sklearn'

### Feature Scaling

In [74]:
from sklearn.preprocessing import StandardScaler
num_scalar = StandardScaler()

ModuleNotFoundError: No module named 'sklearn'

### Pipeline

In [75]:
from sklearn.pipeline import Pipeline,FeatureUnion
num_pipeline = Pipeline([('selector',num_selector),('imputer',num_imputer),('scalar',num_scalar)])
cat_pipeline = Pipeline([('selector',cat_selector),('imputer',cat_imputer),('encoder',cat_encoder)])
full_pipeline = FeatureUnion(transformer_list=[('num_pipeline',num_pipeline),('cat_pipeline',cat_pipeline)])

ModuleNotFoundError: No module named 'sklearn'

### Splitting the data

In [76]:
data_set_X, data_set_Y = splitter.split_from_labels(data_set,label_names)

training_set, validation_set, test_set = splitter.split_tvt_set(data_set,[4,0,1],seed)
training_set_X, training_set_Y = splitter.split_from_labels(training_set,label_names)
test_set_X, test_set_Y = splitter.split_from_labels(test_set,label_names)

training_set_X_trans = full_pipeline.fit_transform(training_set_X)
test_set_X_trans = full_pipeline.fit_transform(test_set_X)

NameError: name 'splitter' is not defined

## Logistic regression

In [77]:
model = LogisticRegression(solver = 'liblinear', random_state=0)
#Will change parameters here according to data analysis
model.fit(training_set_X_trans, training_set_Y) 
y_pred = model.predict(test_set_X_trans)
print(model.score(test_set_X_trans, test_set_Y))

NameError: name 'LogisticRegression' is not defined

## Support Vector Machine



In [78]:
svm = SVC()
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [0.01,0.03,0.035,0.04,0.1,0.3,'auto','scale'],
                     'C': [60,65,70,80]}]
gd = GridSearchCV(svm,scoring='accuracy',param_grid=tuned_parameters)
gd.fit(training_set_X_trans,training_set_Y)
print(gd.best_params_)
gd.best_score_


NameError: name 'SVC' is not defined

# Yang's Solution

In [79]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier, Perceptron
from sklearn import svm
from sklearn.neural_network import MLPClassifier

#randomforest

round1a_rf = RandomForestClassifier(random_state = 1)
round1a_rf.fit(training_set_X_trans,training_set_Y)
a = round1a_rf.predict(test_set_X_trans)

#ridgeclassification
round1a_rc = RidgeClassifier(random_state=1)
round1a_rc.fit(training_set_X_trans,training_set_Y)
b = round1a_rc.predict(test_set_X_trans)

#svm
round1a_svm = svm.SVC(random_state=1)
round1a_svm.fit(training_set_X_trans, training_set_Y)
c = round1a_svm.predict(test_set_X_trans)


#MLPclassifier
round1a_mlp = MLPClassifier(random_state=1)
round1a_mlp.fit(training_set_X_trans, training_set_Y)
d = round1a_mlp.predict(test_set_X_trans)

#Perceptron
round1a_perc = Perceptron(random_state=1)
round1a_perc.fit(training_set_X_trans, training_set_Y)
e = round1a_perc.predict(test_set_X_trans)
#print(e)
#voting
vote = (((a+b+c+d+e)/5) > .5).astype(int)
print(len(vote), '\n', type(vote))
#votedf = pd.DataFrame(data=vote, columns = ['Survived'])
#votedf.to_excel('predictions.xlsx')


#print(vote.score(test_set_X_trans, test_set_Y))

ModuleNotFoundError: No module named 'sklearn'

In [80]:
count = 0
#print(type(y_pred), y_pred.shape)
#print(type(test_set_Y), test_set_Y.shape)

#print(test_set_Y.head())
#for i in range(5):
    #print(test_set_Y[''][i])
arrayedtest = np.array(test_set_Y['Survived'])
#print(arrayedtest, type(arrayedtest))


for i in range(len(vote)):
    #print(y_pred[i], 'y_pred\n')
    #print(arrayedtest[i], 'test_set\n')
    if arrayedtest[i] == vote[i]:
        count = count + 1
print(count/len(vote))

#print(type(test_set_Y))

NameError: name 'test_set_Y' is not defined

# Final Classifier:

In [81]:

round1a_rf = RandomForestClassifier(random_state = 1)
round1a_rf.fit(training_set_X_trans,training_set_Y)
a = round1a_rf.predict(training_set_X_trans)

#ridgeclassification
round1a_rc = RidgeClassifier(random_state=1)
round1a_rc.fit(training_set_X_trans,training_set_Y)
b = round1a_rc.predict(training_set_X_trans)

#svm
round1a_svm = svm.SVC(random_state=1)
round1a_svm.fit(training_set_X_trans, training_set_Y)
c = round1a_svm.predict(training_set_X_trans)


#MLPclassifier
round1a_mlp = MLPClassifier(random_state=1)
round1a_mlp.fit(training_set_X_trans, training_set_Y)
d = round1a_mlp.predict(training_set_X_trans)

#Perceptron
round1a_perc = Perceptron(random_state=1)
round1a_perc.fit(training_set_X_trans, training_set_Y)
e = round1a_perc.predict(training_set_X_trans)

five_results = np.concatenate((a.reshape(-1,1),b.reshape(-1,1),c.reshape(-1,1),d.reshape(-1,1),e.reshape(-1,1)),axis=1)


print(five_results.shape)
print(training_set_Y.shape)
finalClassifier = MLPClassifier(random_state=1)
gd = GridSearchCV(finalClassifier,scoring='accuracy', param_grid={})
gd.fit(five_results,training_set_Y)




a = round1a_rf.predict(test_set_X_trans)
b = round1a_rc.predict(test_set_X_trans)
c = round1a_svm.predict(test_set_X_trans)
d = round1a_mlp.predict(test_set_X_trans)
e = round1a_perc.predict(test_set_X_trans)

pre_pred = np.concatenate((a.reshape(-1,1),b.reshape(-1,1),c.reshape(-1,1),d.reshape(-1,1),e.reshape(-1,1)),axis=1)
arrayedtest = np.array(test_set_Y['Survived'])
#print(arrayedtest, type(arrayedtest))

gd.score(pre_pred,arrayedtest)


NameError: name 'RandomForestClassifier' is not defined

In [82]:
from sklearn.preprocessing import MaxAbsScaler



model = LogisticRegression(solver = 'liblinear', random_state=0)
#Will change parameters here according to data analysis
model.fit(training_set_X_trans, training_set_Y) 
y_pred = model.predict(test_set_X_trans)
print('Unscaled score: ', model.score(test_set_X_trans, test_set_Y))

scaler = MaxAbsScaler(False)
scaler.fit(training_set_X_trans)

X_train = scaler.transform(training_set_X_trans)
X_test = scaler.transform(test_set_X_trans)

model = LogisticRegression(solver = 'liblinear', random_state=0)
#Will change parameters here according to data analysis
model.fit(X_train, training_set_Y) 
y_pred = model.predict(X_test)
print('Scaled score: ', model.score(X_test, test_set_Y))



from sklearn.linear_model import LogisticRegression
Pipeline([('classifier' , LogisticRegression())])
model1 = LogisticRegression()
param_grid = [{'C': [10.0, 100.0, 1000.0], 'gamma': [0.001, 0.0001]]
gd1 = GridSearchCV(model1,scoring='accuracy', param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)
gd1.fit(training_set_X_trans,training_set_Y)
print(gd.best_params_)
gd1.best_score_

SyntaxError: closing parenthesis ']' does not match opening parenthesis '{' (<ipython-input-82-7a06bd5f4b01>, line 28)

TypeError: get_params() missing 1 required positional argument: 'self'

In [83]:
print('e')

e


In [84]:
optimisation = GridSearchCV()
parameters = {'solver':  [‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’, ‘saga’], 'C': [1,2,3,300,500], 'penalty': [‘l1’, ‘l2’, ‘elasticnet’, ‘none’]}
search_model = optimisation(LogisticRegression(), parameters, scoring = 'accuracy')
search_model.fit(training_set_X_trans, training_set_Y)
y_pred_acc = search_model.predict(test_set_X_trans)
print(model.score(test_set_X_trans, test_set_Y))

SyntaxError: invalid character in identifier (<ipython-input-84-f187021f6ed7>, line 2)